In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
import sys
sys.path.append('/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark')

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import glob
import os
import os.path as op
import random
import time
import warnings
warnings.filterwarnings("ignore")

import h5py
from joblib import Parallel, delayed
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

from mri.numerics.gradient import GradAnalysis2

from data import ZeroFilled2DSequence, MaskShifted2DSequence
from evaluate import METRIC_FUNCS, Metrics
from fourier import FFT2
from utils import gen_mask, keras_psnr, keras_ssim
from reconstruction import reco_unet_from_test_file, reco_and_gt_unet_from_val_file, reco_and_gt_unet_from_val_file_no_norm

Using TensorFlow backend.



                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=0.18.0  - required | 1.3.0     installed
numpy          : >=1.11.0  - required | 1.16.4    installed
matplotlib     : >=2.0.0   - required | 3.1.0     installed
future         : >=0.16.0  - required | 0.17.1    installed
astropy        : ==2.0.8   - required | 2.0.8     installed
nibabel        : >=2.1.0   - required | 2.4.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.1.5   - requi

In [4]:
np.random.seed(0)

In [5]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [6]:
fourier_op = FFT2(np.array([1]))

In [7]:
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
val_reco_path = '/media/Zaccharie/UHRes/singlecoil_val_reco/'
val_filenames = glob.glob(val_path + '*')

In [8]:
AF = 8

In [9]:
val_gen = ZeroFilled2DSequence(val_path, mode='validation', af=AF, norm=True)

In [10]:
def metrics_for_args(model):
    metrics = Metrics(METRIC_FUNCS)
    pred_and_gt = [
        reco_and_gt_unet_from_val_file(*val_gen[i], model) 
#         reco_and_gt_unet_from_val_file_no_norm(*val_gen[i], model) 
        for i in tqdm_notebook(range(len(val_gen)), desc='Val files')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Stats'):
        metrics.push(images, im_recos)
    return metrics

In [11]:
run_id = f'unet_wo_lastrelu_af{AF}_test'
epoch = 100
chkpt_path = f'checkpoints/{run_id}-{epoch}.hdf5'
unet = load_model(chkpt_path, custom_objects={'keras_psnr': keras_psnr,'keras_ssim': keras_ssim})

W0801 18:11:27.556612 140211092416256 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0801 18:11:27.677161 140211092416256 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0801 18:11:27.703807 140211092416256 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0801 18:11:27.773637 140211092416256 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastm

In [12]:
metrics = metrics_for_args(unet)

In [13]:
metrics

PSNR = 29.54 +/- 5.065 SSIM = 0.6436 +/- 0.2905

In [14]:
val_gen_eval = ZeroFilled2DSequence(val_path, mode='training', af=AF, norm=True)

metric_keras = unet.evaluate_generator(
    val_gen_eval,
    steps=199,
    max_queue_size=50,
    use_multiprocessing=True,
    workers=35,
    verbose=1,
)

199/199 [==============================] - 53s 268ms/step


In [15]:
metric_keras

[0.37364032334519504,
 0.4202977990782503,
 32.79954080508151,
 0.6665604344513671]